# Calculating Scheduled Headways for CTA Buses - work in progress

### What This Does

TBD

## In progress

- Calculate scheduled headways at each stop, making sure they make sense given different patterns that
run at different times of day for each route


## To Do Next

TBD

## Notes

TBD


## Strategy for Scheduled Headways (Tentative)

1. Which service IDs serve a given stop?

2. When are the earliest/latest times those service IDs are in effect?

3. What are the headways within each timeframe?

## Needs Fixing
 
TBD

In [1]:
import requests
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString
import datetime as dt
import numpy as np
from static_gtfs_analysis import *

# Get Scheduled Stop Times and Headways


In [2]:
# Use Laurie's code to get gtfs feed data
gtfs_feed = download_extract_format('20230105')

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
Loading shapes.txt: 100%|██████████| 7/7 [00:05<00:00,  1.38it/s]


In [3]:
%%capture --no-display
# turn off warnings

######### Scratch
gtfs_version_id = '20230105'
route_id = '55'
service_date_string = '2023-01-09'
# start_timedelta_string ='02:30'
# end_timedelta_string = '26:30'


In [4]:
# Use Laurie's code to get gtfs feed data
gtfs_feed = download_extract_format(gtfs_version_id)    

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
Loading shapes.txt: 100%|██████████| 7/7 [00:04<00:00,  1.41it/s]


In [5]:
def string_to_datetime(date_string:str) -> pendulum.datetime:
        '''Parameters:\n
        date_string is in the format "YYYY-MM-DD" obtained using get_headways().\n
        Data returned:\n
        specified date as a datetime object.'''
        year = int(date_string[:4])
        month = int(date_string[5:7])
        day = int(date_string[8:])
        return pendulum.datetime(year, month, day)


In [6]:
def get_stop_details(gtfs_feed, route_id, service_date_string):

    service_date = string_to_datetime(service_date_string)

    # Get trip summary for service date using chn-ghost-buses make_trip_summary() function
    trip_summary = make_trip_summary(gtfs_feed, service_date, service_date)

    # filter down to the specified route
    trip_summary = trip_summary[trip_summary['route_id'] == route_id]

    # list trip ids for this route
    trip_list = trip_summary['trip_id'].unique().tolist()

    # get stop times data for the trips on this route
    stop_times = gtfs_feed.stop_times

    # filter stop times down to the relevant trips
    stop_times = stop_times.loc[stop_times['trip_id'].isin(trip_list)]

    # Add service id, route, and direction to the stop times data
    stop_times = stop_times.merge(trip_summary[['trip_id', 'route_id', 'service_id', 'direction', 'raw_date']], on='trip_id')

    # filter stop details down to the relevant route
    stop_times = stop_times.loc[stop_times['route_id'] == route_id]

    # Eliminate duplicates - every line shows up twice.
    # TODO:  Investigate why. Is this related to the calendar_cross line in the 
    # make_trip_summary() function? And/or the fact that I'm using the 
    # same date as start and end date as arguments in make_trip_summary?()
    stop_times = stop_times.drop_duplicates()

    # add stop time as a timestamp
    stop_times['stop_time'] = stop_times['raw_date'] + pd.to_timedelta(stop_times['arrival_time'])

    return stop_times

stop_details = get_stop_details(gtfs_feed, route_id, service_date_string)
stop_details

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,route_id,service_id,direction,raw_date,stop_time
0,6530000010020,04:37:30,04:37:30,10603,1,St Louis,0,0,4,4,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-09 04:37:30+00:00
1,6530000010020,04:37:55,04:37:55,10604,2,St Louis,0,639,4,4,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-09 04:37:55+00:00
2,6530000010020,04:38:31,04:38:31,10605,3,St Louis,0,1375,4,4,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-09 04:38:31+00:00
3,6530000010020,04:39:01,04:39:01,10606,4,St Louis,0,2061,4,4,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-09 04:39:01+00:00
4,6530000010020,04:39:32,04:39:32,10607,5,St Louis,0,2826,4,4,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-09 04:39:32+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26368,6530045537030,23:38:09,23:38:09,15366,63,Midway Orange Line,0,45089,23,23,55,65302,West,2023-01-09 00:00:00+00:00,2023-01-09 23:38:09+00:00
26369,6530045537030,23:38:42,23:38:42,10632,64,Midway Orange Line,0,45760,23,23,55,65302,West,2023-01-09 00:00:00+00:00,2023-01-09 23:38:42+00:00
26370,6530045537030,23:39:14,23:39:14,10633,65,Midway Orange Line,0,46457,23,23,55,65302,West,2023-01-09 00:00:00+00:00,2023-01-09 23:39:14+00:00
26371,6530045537030,23:39:44,23:39:44,17030,66,Midway Orange Line,0,47108,23,23,55,65302,West,2023-01-09 00:00:00+00:00,2023-01-09 23:39:44+00:00


In [23]:
stop_details.loc[stop_details['stop_id'] == '14122']

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,route_id,service_id,direction,raw_date,stop_time
152,6530000057020,17:07:00,17:07:00,14122,67,Midway Orange Line,0,52206,17,17,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-09 17:07:00+00:00
398,6530000586020,24:24:30,24:24:30,14122,67,Midway Orange Line,0,52206,0,0,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-10 00:24:30+00:00
400,6530000814020,13:23:30,13:23:30,14122,1,Museum of Science & Industry,0,0,13,13,55,65301,East,2023-01-09 00:00:00+00:00,2023-01-09 13:23:30+00:00
638,6530000754020,08:43:00,08:43:00,14122,67,Midway Orange Line,0,52206,8,8,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-09 08:43:00+00:00
804,6530001034020,07:08:00,07:08:00,14122,67,Midway Orange Line,0,52206,7,7,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-09 07:08:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25647,6530044324020,17:48:00,17:48:00,14122,1,Museum of Science & Industry,0,0,17,17,55,65301,East,2023-01-09 00:00:00+00:00,2023-01-09 17:48:00+00:00
25957,6530044871030,20:57:00,20:57:00,14122,67,Midway Orange Line,0,52206,20,20,55,65302,West,2023-01-09 00:00:00+00:00,2023-01-09 20:57:00+00:00
26014,6530046007020,09:59:00,09:59:00,14122,1,Museum of Science & Industry,0,0,9,9,55,65301,East,2023-01-09 00:00:00+00:00,2023-01-09 09:59:00+00:00
26304,6530045217020,24:03:30,24:03:30,14122,67,Midway Orange Line,0,52206,0,0,55,65301,West,2023-01-09 00:00:00+00:00,2023-01-10 00:03:30+00:00


In [33]:
%%capture --no-display

# Find times when ANY buses on any service for this route and direction are running at a given 
# bus stop.  This will let us skip out-of-service times in the headway calcs, so those won't 
# show up as long headways.

def get_active_service_times(stop_details, stop_id, direction):

    # dataframe to contain service time ranges
    service_ranges = pd.DataFrame()

    # filter stop details to a single stop and direction of travel
    single_stop_details = stop_details.loc[
        stop_details['stop_id'] == stop_id].loc[
            stop_details['direction'] == direction]
    
    # find service IDs that serve the stop
    service_ids = set(single_stop_details['service_id'])

    # find times when each service starts and ends
    for service_id in service_ids:
        df_service = single_stop_details.loc[single_stop_details['service_id'] == service_id]
        times = df_service['stop_time'].tolist()
        start_time = min(times)
        end_time = max(times)

        # make a single-row dataframe containing start and end times of 
        # one service id at this stop
        df = pd.DataFrame(
            [[service_id, start_time, end_time]],
            columns=['service_id', 'start_time', 'end_time'])

        # Add this service id's time range to the dataframe for all service id's time ranges
        service_ranges = pd.concat([service_ranges, df])

        # sort ranges by start time
        service_ranges.sort_values('start_time', inplace=True)

        # list the start time of the next range
        service_ranges['next_start_time'] = np.roll(service_ranges['start_time'].tolist(), shift=-1,)

        # remove next start time from the last line
        service_ranges['next_start_time'].iloc[-1] = None


    # reset the index
    service_ranges.reset_index(inplace=True)

    # generate a list of start and end times when ANY service is active.
    active_service_times = []
    start_time = 0
    end_time = 0

    # iterate through all service id's time ranges
    for idx, row in service_ranges.iterrows():

        # last range:  set the end time for the current in-service
        # range and add in-service range to the list.
        if idx == len(service_ranges) - 1:
            end_time = row['end_time']
            active_service_times.append((start_time, end_time))

        else:

            #  if there was no service before this range began:
            #  start a new in-service time range with a new start time
            if start_time == 0:
                start_time = row['start_time']


            # if there is a service gap after this range: 
            # this service's end time is the end of the overall in-service time.
            # End the in-service range and add it to the list.
            if row['next_start_time'] > row['end_time']:
                end_time = row['end_time']
                active_service_times.append([start_time, end_time])
                start_time = 0


    return active_service_times

 
get_active_service_times(stop_details, '14122', 'East')

[[Timestamp('2023-01-09 00:05:30+0000', tz='Timezone('UTC')'),
  Timestamp('2023-01-09 02:01:00+0000', tz='Timezone('UTC')')],
 (Timestamp('2023-01-09 05:00:00+0000', tz='Timezone('UTC')'),
  Timestamp('2023-01-09 23:42:30+0000', tz='Timezone('UTC')'))]

In [41]:
%%capture --no-display

# Get headways
def get_headways(stop_details, stop_id, direction):
    
    df_headways = pd.DataFrame()

    active_service_times = get_active_service_times(stop_details, stop_id, direction)

    # filter to a single stop id and direction of travel
    # for stop_id, direction in set(zip(stop_details['stop_id'], stop_details['direction'])):

    # dataframe filtered to one stop_id and direction
    df = stop_details.loc[
        stop_details['stop_id'] == stop_id].loc[
            stop_details['direction'] == direction]

    # sort by arrival time
    df = df.sort_values('arrival_time')

    # add previous stop times to each line
    df['previous_stop_time'] = np.roll(df['stop_time'], shift=1)

    # Calculate headways
    df['headway'] = df['stop_time'] - df['previous_stop_time']

    # Remove the first headway and previous stop time for the first
    # bus in each active service time range (no previous arrival time to compare with)
    active_service_starts = [start for (start, end) in active_service_times]

    start_filter = df['stop_time'].isin(active_service_starts)

    df.loc[start_filter, 'headway'] = None
    df.loc[start_filter, 'previous_stop_time'] = None

    return df
    

headways = get_headways(stop_details, '14122', 'East').head(50)



In [42]:
# Copied from actual_headways.ipynb

def get_headway_stats(headways:pd.DataFrame, headway_column_name:LineString) -> dict:
    '''Parameters:\n
    headways is a dataframe obtained using get_headways() or get_scheduled_headways().\n
    headway_column_name is the name of the column containing headways:  'est_headway' if these
    are based on actual bus times using get_headways() or 'scheduled_headway' if these are based
    on GTFS schedules using get_Scheduled_headways()\n
 
    Data returned:\n
    Statisics on the headways are returned as a dictionary.'''
    est_headways = headways[headway_column_name]
    stats = {
        'mean':est_headways.mean(),
        'max':est_headways.max(),
        'min':est_headways.min(),
        '25th_percentile':est_headways.quantile(0.25), # 25th percentile
        'median':est_headways.median(), # 50th percentile
        '75th_percentile':est_headways.quantile(0.75)
    }
    return stats

get_headway_stats(headways, 'headway')

{'mean': Timedelta('0 days 00:14:50.625000'),
 'max': Timedelta('0 days 00:30:00'),
 'min': Timedelta('0 days 00:07:00'),
 '25th_percentile': Timedelta('0 days 00:12:00'),
 'median': Timedelta('0 days 00:12:00'),
 '75th_percentile': Timedelta('0 days 00:15:52.500000')}